In [1]:
RANDOM_STATE = 101
no_dummies = False # 'no dummies' in DATA_DETAIL
VERSION = '09'

AUTOENCODE_EPOCHS = 400
AUTOENCODE_EPOCHS = 80
#AUTOENCODE_EPOCHS = 10

In [2]:
import sys
import os
import json

module_path = os.path.abspath(os.path.join('..', '..', '..'))
if module_path not in sys.path:
    #sys.path.append(module_path+"\\zfunctions")
    sys.path.append(module_path)

#with open('../../z_envs/_envs.json') as f:
#    env_vars = json.loads(f.read())

try:
    import google.colab

    run_env = 'colab'
except:
    try:
        run_env = env_vars['notebook_environment']
    except:
        run_env = 'unknown'

if "JPY_PARENT_PID" in os.environ:
    is_jupyter = True
else:
    is_jupyter = False



if run_env not in ['colab', 'gradient', 'cloud','unknown']:
    cloud_run = False
    from functions_b__get_the_data_20221116 import set_csv_directory
    set_csv_directory('final_split')
else:
    cloud_run = True

from functions_0__common_20221116 import get_columns
from functions_b__get_the_data_20221116 import get_combined_dataset, get_source_dataframe
from functions_d1__prepare_cleanse_data_20221116 import tidy_dataset
from functions_d2__transform_enrich_data_20221116 import preprocess, feature_engineer
from functions_d3__prepare_store_data_20221116 import create_train_test_data
from functions_e__train_model_20221116 import get_chosen_model, make_modelling_pipeline, get_cv_params, fit_model_with_cross_validation, get_hyperparameters
from functions_f_evaluate_model_20221116 import get_best_estimator_average_time, get_results, update_results

#print(env_vars)

ModuleNotFoundError: No module named 'functions_0__common_20221116'

In [ ]:
columns, booleans, floats, categories, custom, wildcard = get_columns(version=VERSION)
LABEL = 'Price'

In [ ]:
df, retrieval_type = get_source_dataframe(cloud_run, VERSION, folder_prefix='../../../', row_limit=None)
df_orig = df.copy()

if retrieval_type != 'tidy':
    df = tidy_dataset(df, version=int(VERSION))
    df = feature_engineer(df, version=int(VERSION))


    df = df[columns]

In [ ]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig, X_train_index, X_test_index, y_train_index, y_test_index, df_features, df_labels = create_train_test_data(
    df,
    categories=categories,
    RANDOM_STATE=RANDOM_STATE, return_index=True,
    drop_nulls=True,
    no_dummies=no_dummies
)

#print(X_train[0])
print(df.shape)
print(X_train_orig.shape, X_test_orig.shape, y_train_orig.shape, y_test_orig.shape, X_train_index.shape, X_test_index.shape,
      y_train_index.shape, y_test_index.shape)



In [ ]:
X_train = X_train_orig.copy()
X_test = X_test_orig.copy()

In [ ]:
from sklearn.svm import SVR
from keras.saving.save import load_model

# load the model from file
encoder = load_model(f'autoencoder_encoder_v{VERSION}.h5')

# encode the train data
X_train_encode = encoder.predict(X_train)
# encode the test data
X_test_encode = encoder.predict(X_test)

In [ ]:
X_train_orig[0:3]

In [ ]:
X_train[0:3]

In [ ]:
X_test_encode[0:3]

In [ ]:
y_train = y_train_orig
y_test = y_test_orig

In [ ]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor

# define model
#model = SVR()
#model = LinearRegression()
model = KNeighborsRegressor()

# fit model on the training dataset
model.fit(X_train_encode, y_train)
# make prediction on test set

In [ ]:
y_pred = model.predict(X_test_encode)
y_pred_orig = y_pred

In [ ]:
# invert transforms so we can calculate errors
y_pred = y_pred.reshape((len(y_pred), 1))


#trans_out = MinMaxScaler()
#trans_out.fit(y_train)

#y_pred = trans_out.inverse_transform(y_pred)
#??? y_test = trans_out.inverse_transform(y_test)
# calculate error

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
mae_score = mean_absolute_error(y_test, y_pred)
score = r2_score(y_test, y_pred)
print(mae_score)
print(score)

In [ ]:
print(y_test[0], y_pred[0])


In [ ]:
import numpy as np

print("Score (mean absolute error):", mean_absolute_error(y_test, y_pred), "\n")
compare_np = np.hstack((y_test, y_pred))
compare_df = pd.DataFrame(compare_np, columns=['actual', 'predicted'])
compare_df['diff 1 %'] = abs((compare_df['actual'] - compare_df['predicted']) / compare_df['actual'] * 100)
compare_df['diff 2 %'] = abs((compare_df['actual'] - compare_df['predicted']) / compare_df['predicted']) * 100

compare_df

In [ ]:
import matplotlib.pyplot as plt
best_model_fig, best_model_ax = plt.subplots()
best_model_ax.scatter(y_test, y_pred, edgecolors=(0, 0, 1))
best_model_ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
best_model_ax.set_ylabel('Predicted')
best_model_ax.set_xlabel('Actual')
#ax.title.set_text(f'CV Chosen best option ({calculated_best_pipe[1]})')

plt.show()